In [1]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LEARNING_RATE = 1e-5

In [2]:
### training set 있는 Path ###
BASE_PATH = '../data/train/'

In [3]:
import torchvision.transforms as transforms
### Transforms ###
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(300)
])

## Load dataset

In [4]:
from Dataset import TrainValidDataset

In [5]:
import pandas as pd
df = pd.read_csv(BASE_PATH + 'train.csv')

In [6]:
# training set (validation set 따로 안 나눈 경우)
# train_dataset = AllClassDataset(
#     base_path = BASE_PATH, 
#     data = df, 
#     transform = transform
# )

In [7]:
# train / validation data
from sklearn.model_selection import train_test_split
def mapAgeGender(age, gender):
    answer = 0
    if age < 30:
        answer += 0
    elif age >= 60:
        answer += 2
    else:
        answer += 1
    return answer if gender == 'male' else answer + 3

y_data = df.apply(lambda x: mapAgeGender(x['age'], x['gender']), axis=1)
x_train, x_val, y_train, y_val = train_test_split(df.index, y_data, test_size=0.2, random_state=42, stratify=y_data)

In [8]:
gender_dataset = TrainValidDataset(
    base_path = BASE_PATH, 
    data = df.loc[x_train], 
    transform = transform,
    label="gender"
)

100%|██████████| 2160/2160 [00:17<00:00, 125.13it/s]


In [9]:
gender_val_dataset = TrainValidDataset(
    base_path = BASE_PATH, 
    data = df.loc[x_val], 
    transform = transform,
    label="gender"
)

100%|██████████| 540/540 [00:03<00:00, 140.47it/s]


### Dataset 확인용 Code
```python
import matplotlib.pyplot as plt
image, label = val_dataset[0]
plt.title(f'Class {label}')
plt.imshow(image.permute(1,2,0))
plt.show()
```

## DataLoader

In [10]:
gender_trainloader = torch.utils.data.DataLoader(
    gender_dataset,
    batch_size=64,
    num_workers=1
)

In [11]:
gender_valloader = torch.utils.data.DataLoader(
    gender_val_dataset,
    batch_size=64,
    num_workers=1
)

### Dataloader 확인용 Code
```python
import matplotlib.pyplot as plt
images, labels = next(iter(trainloader))
plt.title(f'Class {labels[0]}')
plt.imshow(images[0].permute(1,2,0))
plt.show()
```

## Model

In [12]:
from Model import ModifiedModel

In [13]:
gender_model = ModifiedModel(num_classes = 2).to(DEVICE)

In [14]:
gender_optimizer = torch.optim.Adam(gender_model.parameters(), lr=LEARNING_RATE)

## Train

In [15]:
from train import train

In [16]:
# Tensorboard
from torch.utils.tensorboard import SummaryWriter
train_writer = SummaryWriter('runs/train_resnet18_gender')
val_writer = SummaryWriter('runs/val_resnet18_gender')

In [17]:
### Parameter 변경 ###
train(
    model=gender_model, 
    optimizer=gender_optimizer,
    train_loader=gender_trainloader, 
    val_loader=gender_valloader,
    device=DEVICE, 
    epochs=30, 
    save=True, 
    saved_folder="saved/gender",
    train_writer=train_writer,
    val_writer=val_writer
)

  0%|          | 0/237 [00:00<?, ?it/s]

Epoch 001: Loss: 0.3261 / Acc: 0.860 / F1: 0.85        | Val Loss: 0.1190 / Val Acc: 0.963 / Val F1: 0.95


  0%|          | 0/237 [00:00<?, ?it/s]

Epoch 002: Loss: 0.1287 / Acc: 0.957 / F1: 0.95        | Val Loss: 0.0817 / Val Acc: 0.972 / Val F1: 0.96


  0%|          | 0/237 [00:00<?, ?it/s]

...

Epoch 028: Loss: 0.0007 / Acc: 1.000 / F1: 1.00        | Val Loss: 0.0540 / Val Acc: 0.983 / Val F1: 0.98


  0%|          | 0/237 [00:00<?, ?it/s]

Epoch 029: Loss: 0.0006 / Acc: 1.000 / F1: 1.00        | Val Loss: 0.0544 / Val Acc: 0.983 / Val F1: 0.98


100%|██████████| 237/237 [01:03<00:00,  3.76it/s]


Epoch 030: Loss: 0.0005 / Acc: 1.000 / F1: 1.00        | Val Loss: 0.0548 / Val Acc: 0.983 / Val F1: 0.98


In [ ]:
train_writer.close()
val_writer.close()

In [2]:
# rm -rf /opt/ml/.cache/torch/hub/checkpoints/